In [7]:
import pandas as pd
import tensorflow as tf
from Preprocessing.to_embedding import WordEmbedding
from Preprocessing.data_format import formatting
from sklearn.model_selection import train_test_split

In [8]:
data = formatting("phase1_movie_reviews-train.csv")
y = data[['polarity']]

In [9]:
embedding = WordEmbedding(num_features = 300)
WordEmbedding.fit(embedding, data['reviewText'])

In [10]:
WordEmbedding.size(embedding)

Total number of words in the vocabulary:  (54844, 300)


In [11]:
train_embeddings = WordEmbedding.to_pd(embedding, data['reviewText'])

In [12]:
train_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.271564,0.037601,0.554395,0.194177,-0.295273,0.413215,-0.033724,0.302937,0.749933,0.580825,...,-0.182048,0.331851,0.119295,-0.271650,0.120633,0.254109,-0.141262,-0.199510,0.211353,-0.284701
1,-0.393395,-0.062188,0.635279,0.062630,-0.144905,0.324336,-0.142172,0.261326,0.665856,0.308496,...,-0.190262,0.190294,-0.000945,0.049691,0.039310,0.229791,-0.263687,-0.117109,0.007234,-0.407619
2,-0.229396,-0.098869,0.327320,0.247322,0.011974,0.259872,0.081208,0.320640,0.435121,0.051939,...,-0.027018,0.114487,-0.116905,0.148160,0.144647,-0.113864,-0.233946,-0.004275,-0.230247,-0.201205
3,-0.211428,-0.095357,0.239634,0.099847,-0.106311,0.255665,-0.000272,0.178190,0.364575,0.101936,...,0.033259,0.137756,-0.125509,-0.079346,0.033270,0.086103,-0.125738,-0.139738,-0.103189,-0.318689
4,-0.578025,0.039751,0.582209,0.288721,-0.404956,0.231579,0.051184,0.170111,0.673649,0.285740,...,-0.353686,0.410463,0.154479,-0.248465,0.176782,0.508400,-0.220848,-0.341107,0.029526,-0.497928


In [13]:
data = data[['year']]
data = pd.concat([data, train_embeddings], axis=1, sort=False)

In [14]:
y = pd.get_dummies(y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.20, random_state = 42)

In [13]:
x = tf.placeholder(tf.float32, [None, X_train.shape[1]])
t = tf.placeholder(tf.float32, [None, y_train.shape[1]])

#Defining model parameters:
W1 = tf.get_variable("W1", [X_train.shape[1], 3], initializer=tf.random_normal_initializer)
b1 = tf.get_variable("b1", [3], initializer=tf.random_normal_initializer)
W2 = tf.get_variable("W2", [3, y_train.shape[1]], initializer=tf.random_normal_initializer)
b2 = tf.get_variable("b2", [y_train.shape[1]], initializer=tf.random_normal_initializer)

#Construct model:
z1 = tf.nn.relu(tf.matmul(x, W1) + b1)
y = tf.matmul(z1, W2) + b2

# Variables for prediction and accuracy
prediction = tf.argmax(y, 1)

accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(t, 1)), tf.float32))

# Difine the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=t, logits=y))

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0001).minimize(loss)

In [16]:
# Make an operation that initializes the variables
init = tf.global_variables_initializer()

# Start a new session
with tf.Session() as session:
    # Initialize the values
    session.run(init)
    
    # Training cycle
    for epoch in range(30000):
        _, loss_value = session.run([optimizer, loss], feed_dict={x: X_train, t: y_train})
        
        if epoch % 1000 == 0:
            print("Epoch: ", epoch, "loss =", loss_value)            
            
    print("Optimization done")

    # Evaluate the accuracy on the test set
    accuracy_value = session.run(accuracy, feed_dict={x: X_train, t: y_train})
    print("Accuracy:", accuracy_value)
    accuracy_value = session.run(accuracy, feed_dict={x: X_test, t: y_test})
    print("Accuracy:", accuracy_value)

Epoch:  0 loss = 623.6992
Epoch:  1000 loss = 0.6936612
Epoch:  2000 loss = 0.6936578


KeyboardInterrupt: 